In [ ]:
from dataset.dataset_utils import generate_all
generate_all()


In [7]:
from imitation.data.types import Trajectory
import numpy as np

# Let's load an expert sample:
import json
  

# This function will load a trajectory (one episode):
def make_trajectory(key):
    observations = []
    actions = []
    infos = []
    for i in range(9):
        # Opening JSON file
        f = open('dataset/trajectories/data_{}_{}.json'.format(key,i))
        data = json.load(f)




        obs = np.concatenate(( np.array(data['conditions']),
                            np.array([data['volume']]),
                            np.array([data['n_steps_left']]),
                            np.array(data['design_variables']).squeeze(),
        ))
                            

        acts = np.array(data['action'])
        
        observations.append(obs)
        infos.append({})
        
        if acts != []:
            actions.append(acts.squeeze())

        terminal = True
    
    return np.array(observations),np.array(actions),terminal,infos

trajectories = []
for z in range(7500):
    try:
        obs,acts,terminal,infos = make_trajectory(z)
        trajectories.append(Trajectory(obs=obs,acts=acts,terminal=terminal,infos=None))
    except:
        continue

/var/folders/w0/dpd5xh7x2nxg0zjcs7pk1q3w0000gn/T/ipykernel_23563/2195412158.py:33: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if acts != []:
/var/folders/w0/dpd5xh7x2nxg0zjcs7pk1q3w0000gn/T/ipykernel_23563/2195412158.py:33: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if acts != []:


In [13]:
from sogym.env import sogym
train_env = sogym(nelx=100,nely=50,mode='train',observation_type='box_dense')

In [16]:
from stable_baselines3.ppo import MlpPolicy
from imitation.algorithms.adversarial.airl import AIRL
from imitation.rewards.reward_nets import BasicShapedRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize, SubprocVecEnv

from imitation.util import logger as imit_logger

# Create the vectorized environment
num_cpu = 4 # Number of processes to use
venv= make_vec_env(lambda:train_env, n_envs=num_cpu,vec_env_cls=SubprocVecEnv)
#venv=VecNormalize(venv,gamma=1.0,norm_obs=False)

policy_kwargs = dict(
    net_arch=dict(pi=[512,512,512], vf=[512,512,512]),
)

learner = PPO("MlpPolicy",
    env=venv,
    n_steps=8*28,  #def: 224
    policy_kwargs=policy_kwargs,
    batch_size=8*28*1, #def: 50*224
    ent_coef=0.001,
    gamma=1.0,
    learning_rate=1e-3,
    tensorboard_log="test_logs",

)

custom_logger = imit_logger.configure(
        folder="test_logs",
        format_strs=["tensorboard", "stdout"],
    )

In [17]:
reward_net = BasicShapedRewardNet(
    venv.observation_space, venv.action_space,
    reward_hid_sizes=(32,32),
    potential_hid_sizes=(32, 32),
    discount_factor=1.0,
    dropout_prob=0.95
)

#learner=PPO.load('imit',env=venv,device='cuda')

airl_trainer = AIRL(
    demonstrations=trajectories,
    demo_batch_size=2048,
    debug_use_ground_truth=True,
    init_tensorboard=True,
    log_dir="test_logs",
    n_disc_updates_per_round=1,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net,
    custom_logger= custom_logger

)

In [18]:
airl_trainer.train(4000000)  # Note: set to 300000 for better results

round:   0%|          | 0/4464 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 8        |
|    gen/rollout/ep_rew_mean  | 0        |
|    gen/time/fps             | 148      |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 6        |
|    gen/time/total_timesteps | 896      |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.504    |
|    disc/disc_acc_expert             | 0.995    |
|    disc/disc_acc_gen                | 0.0137   |
|    disc/disc_entropy                | 0.0249   |
|    disc/disc_loss                   | 3.63     |
|    disc/disc_proportion_expert_pred | 0.991    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 2.05e+03 |
|    disc/n_generated                 | 2.05e+03 |
-

round:   0%|          | 1/4464 [00:07<9:04:50,  7.32s/it]

--------------------------------------------------
| raw/                              |            |
|    gen/rollout/ep_len_mean        | 8          |
|    gen/rollout/ep_rew_mean        | 7.58e-05   |
|    gen/time/fps                   | 151        |
|    gen/time/iterations            | 1          |
|    gen/time/time_elapsed          | 5          |
|    gen/time/total_timesteps       | 1792       |
|    gen/train/approx_kl            | 0.09491729 |
|    gen/train/clip_fraction        | 0.446      |
|    gen/train/clip_range           | 0.2        |
|    gen/train/entropy_loss         | -8.48      |
|    gen/train/explained_variance   | -40.2      |
|    gen/train/learning_rate        | 0.001      |
|    gen/train/loss                 | -0.0561    |
|    gen/train/n_updates            | 10         |
|    gen/train/policy_gradient_loss | -0.0456    |
|    gen/train/std                  | 0.988      |
|    gen/train/value_loss           | 0.108      |
-------------------------------

round:   0%|          | 2/4464 [00:14<8:55:21,  7.20s/it]

...

-------------------------------------------------
| raw/                              |           |
|    gen/rollout/ep_len_mean        | 8         |
|    gen/rollout/ep_rew_mean        | 1.28e-05  |
|    gen/time/fps                   | 129       |
|    gen/time/iterations            | 1         |
|    gen/time/time_elapsed          | 6         |
|    gen/time/total_timesteps       | 47488     |
|    gen/train/approx_kl            | 0.7731699 |
|    gen/train/clip_fraction        | 0.748     |
|    gen/train/clip_range           | 0.2       |
|    gen/train/entropy_loss         | -7.25     |
|    gen/train/explained_variance   | -2.56     |
|    gen/train/learning_rate        | 0.001     |
|    gen/train/loss                 | -0.115    |
|    gen/train/n_updates            | 520       |
|    gen/train/policy_gradient_loss | -0.0886   |
|    gen/train/std                  | 0.81      |
|    gen/train/value_loss           | 2.37e-05  |
-------------------------------------------------


round:   1%|          | 53/4464 [07:01<9:50:50,  8.04s/it]

--------------------------------------------------
| raw/                              |            |
|    gen/rollout/ep_len_mean        | 8          |
|    gen/rollout/ep_rew_mean        | 0.00044    |
|    gen/time/fps                   | 140        |
|    gen/time/iterations            | 1          |
|    gen/time/time_elapsed          | 6          |
|    gen/time/total_timesteps       | 48384      |
|    gen/train/approx_kl            | 0.93194425 |
|    gen/train/clip_fraction        | 0.75       |
|    gen/train/clip_range           | 0.2        |
|    gen/train/entropy_loss         | -7.21      |
|    gen/train/explained_variance   | -29.1      |
|    gen/train/learning_rate        | 0.001      |
|    gen/train/loss                 | -0.141     |
|    gen/train/n_updates            | 530        |
|    gen/train/policy_gradient_loss | -0.103     |
|    gen/train/std                  | 0.807      |
|    gen/train/value_loss           | 2.91e-05   |
-------------------------------

round:   1%|          | 54/4464 [07:08<9:42:27,  7.92s/it]

-------------------------------------------------
| raw/                              |           |
|    gen/rollout/ep_len_mean        | 8         |
|    gen/rollout/ep_rew_mean        | 0.000309  |
|    gen/time/fps                   | 150       |
|    gen/time/iterations            | 1         |
|    gen/time/time_elapsed          | 5         |
|    gen/time/total_timesteps       | 49280     |
|    gen/train/approx_kl            | 0.8978373 |
|    gen/train/clip_fraction        | 0.758     |
|    gen/train/clip_range           | 0.2       |
|    gen/train/entropy_loss         | -7.18     |
|    gen/train/explained_variance   | -6.94     |
|    gen/train/learning_rate        | 0.001     |
|    gen/train/loss                 | -0.13     |
|    gen/train/n_updates            | 540       |
|    gen/train/policy_gradient_loss | -0.0897   |
|    gen/train/std                  | 0.802     |
|    gen/train/value_loss           | 1.85e-05  |
-------------------------------------------------
